Zunächst müssen wir das OdeNet-Modul importieren

## from odenet import *


Mit OdeNet.word_info() greift man auf Lexikon-Einträge zu, bekommt die Lexikon-ID für ein Wort, den Lemma-Wert, POS und die IDs der Synsets, in denen das Wort enthalten ist.

In [4]:
OdeNet.word_info("Eigenschaft")

Eigenschaft n 
SENSE: odenet-1203-n  ('i35577', [], 'Abstraktion, die eine Entität beschreibt.', [('hypernym', 'odenet-8570-n'), ('hyponym', 'odenet-10789-n'), ('hyponym', 'odenet-4026-n'), ('hyponym', 'odenet-7939-n')], ['Besonderheit', 'Spezifikum', 'Eigenheit', 'Eigentümlichkeit', 'Wesensmerkmal', 'Kennzeichen', 'Charakteristikum', 'Eigenschaft', 'Charakterzug', 'Attribut', 'Attributivum', 'Merkmal', 'Eigenart', 'Manier', 'Duktus', 'Wesenszug'])

SENSE: odenet-4026-n  ('i67612', [], 'Gesamtheit der Eigenschaften einer Person.', [('hypernym', 'odenet-1203-n')], ['Zug', 'Einstellung', 'Eigenschaft', 'Charakterzug', 'Merkmal', 'Haltung', 'Charakteristik', 'Charaktermerkmal', 'Charaktereigenschaft'])

SENSE: odenet-8857-n  ('i75200', [], 'Etwas, das zu einem Ergebnis beiträgt.', [('hypernym', 'odenet-8141-n'), ('hyponym', 'odenet-3739-n'), ('hyponym', 'odenet-460-n')], ['Eigenschaft', 'Umstand', 'Faktor', 'Tatsache', 'Gegebenheit', 'Rahmenbedingung', 'Randbedingung'])

SENSE: odenet-107

Hier bekommt man die Lexikon-IDs für eine Liste von Wörtern.

In [3]:
wordlist = ['Frühling','Sommer','Herbst','Winter']
for w in wordlist:
    print(w + ': ' + OdeNet.word_id(w))

Frühling: w14145
Sommer: w44811
Herbst: w1202374
Winter: w25612


Mit check_word_id bekommt man für eine Lexikon-ID Lemma, POS, Synsets und Relationen

In [4]:
check_word_id('w14145')

LEMMA: Frühling
POS: n
SENSE ID: w14145_3067-n


Mit words_in_synset bekommt man die Wörter, die in einem Synset sind.

In [7]:
words_in_synset('odenet-9673-n')

['Weißkittel',
 'Doktor',
 'Medikus',
 'Arzt',
 'Onkel Doktor',
 'Heilkundiger',
 'Herr Doktor',
 'Mediziner',
 'Halbgott in Weiß',
 'Doc',
 'Ärztin']

Mit check_synset bekommt man alle Informationen zu einem Synset.

In [5]:
#print(check_synset("odenet-3812-n"))
print(check_synset("odenet-48-n"))
print(check_synset("odenet-2640-n"))

('', [], 'Der nicht beschriebene Teil zwischen zwei Wörtern.', [('hypernym', 'odenet-2640-n')], ['Abstand', 'Leerstelle', 'Leerschlag', 'Leerzeichen', 'Leerschritt', 'Wortzwischenraum'])
('i63745', 'the distance by which one thing clears another; the space between them', [], [('hyponym', 'odenet-226-n'), ('hyponym', 'odenet-20472-n')], ['Leerstelle', 'Hohlraum', 'Zwischenraum', 'Lücke', 'Kavität', 'Leerraum', 'Auslassung', 'Leerfeld'])


In [3]:
hypernyms_word("Distanz")

[]

In [8]:
hyponyms_word("Veranstaltung")

[('w64697_17741-n',
  'odenet-14850-n',
  ['Lager',
   'Kinderlandverschickung',
   'Freizeit',
   'Ferienlager',
   'Ferienfreizeit']),
 ('w64697_17741-n',
  'odenet-17740-n',
  ['Auftritt',
   'Gig',
   'Live Event',
   'Live-Act',
   'Live-Veranstaltung',
   'Liveact',
   'Live-Gig',
   'Show',
   'Live-Auftritt',
   'Live-Entertainment']),
 ('w64697_17741-n',
  'odenet-17742-n',
  ['Tiershow', 'Tiertheater', 'Tierveranstaltung']),
 ('w64697_17741-n',
  'odenet-214-n',
  ['Besprechung', 'Consulting', 'Beratung', 'Konsultation', 'Gespräch']),
 ('w64697_17741-n', 'odenet-25683-n', ['Matinee', 'Vormittagsveranstaltung']),
 ('w64697_17741-n',
  'odenet-2801-n',
  ['Gespräch',
   'Austausch',
   'Unterhaltung',
   'Gedankenaustausch',
   'Zwiegespräch',
   'Unterredung',
   'Diskussion',
   'Zwiesprache',
   'Konversation',
   'Wortwechsel',
   'Aussprache',
   'Dialog']),
 ('w64697_17741-n',
  'odenet-32187-n',
  ['Vorkurs',
   'Einführungsveranstaltung',
   'Vorbereitungskurs',
   'Vor

In [9]:
meronyms_word("Morgenland")

[('w12715_2720-n',
  'odenet-14514-n',
  ['Saudi-Arabien', 'Königreich Saudi-Arabien', 'Saudisch-Arabien']),
 ('w12715_2720-n', 'odenet-14519-n', ['Syrien', 'Arabische Republik Syrien']),
 ('w12715_2720-n', 'odenet-14516-n', ['Türkei', 'Republik Türkei']),
 ('w12715_9897-n', 'odenet-9309-n', ['Ferner Osten', 'Fernost'])]

In [10]:
myword = "zeigen"
(lemma_id, lemma_value, pos, senses) = check_word_lemma(myword)
print (lemma_value + " " + pos + " ")
for sense in senses:
    print("SENSE: " + str(sense[1]) + "  " + str(check_synset(sense[1])) + "\n")
print("HYPERNYMS: " + str(hypernyms_word(myword)))
print("HYPONYMS: " + str(hyponyms_word(myword)))
print("MERONYMS: " + str(meronyms_word(myword)))
print("HOLONYMS: " + str(holonyms_word(myword)))
print("ANTONYMS: " + str(antonyms_word(myword)))


zeigen v 
SENSE: odenet-533-v  ('i33216', [], 'Vorschlagen, etwas für jemanden zu tun.', [('hypernym', 'odenet-8644-v')], ['andienen', 'anbieten', 'bieten', 'zeigen'])

SENSE: odenet-3065-v  ('i32498', [], 'Jemanden etwas sehen lassen.', [('hypernym', 'odenet-9246-v')], ['zeigen', 'präsentieren', 'darstellen', 'vorzeigen', 'abbilden', 'vorführen', 'demonstrieren', 'vorstellen'])

SENSE: odenet-4206-v  ('i32440', [], 'Etwas öffentlich sagen.', [('hyponym', 'odenet-2339-v'), ('hyponym', 'odenet-11159-v'), ('hyponym', 'odenet-1192-v'), ('hyponym', 'odenet-22914-v'), ('hyponym', 'odenet-2770-v')], ['kundtun', 'zeigen', 'offenbaren', 'bekunden'])

SENSE: odenet-5328-v  ('i25048', [], 'Die Wahrheit von etwas deutlich machen.', [('hypernym', 'odenet-12792-v')], ['bestätigen', 'zeigen', 'aufzeigen', 'nachweisen', 'belegen', 'Nachweis erbringen', 'Beweis erbringen für', 'beweisen'])

SENSE: odenet-9990-v  ('i25049', [], 'Sich als etwas erweisen, offenbaren.', [], ['zeigen', 'niederschlagen', 'm